In [2]:
import os
from pathlib import Path
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [5]:
from langchain_experimental.open_clip import OpenCLIPEmbeddings

In [12]:
embedding = OpenCLIPEmbeddings(
    model_name="ViT-B-32",
    checkpoint="laion2b_s34b_b79k", 
    device="cpu")

c:\Users\Narala Nithin\OneDrive\Desktop\semantic_image_search\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
c:\Users\Narala Nithin\OneDrive\Desktop\semantic_image_search\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Narala Nithin\.cache\huggingface\hub\models--laion--CLIP-ViT-B-32-laion2B-s34B-b79K. Caching files will still work but in a degraded version that might require more space on your disk. This warning

In [13]:
api_key=os.getenv("QDRANT_API_KEY")
url=os.getenv("API_ENDPOINT")


In [14]:
from qdrant_client import QdrantClient

In [15]:
qdrant_client=QdrantClient(api_key=api_key,url=url)

In [16]:
qdrant_client.get_collections()

CollectionsResponse(collections=[])

In [23]:
collections=qdrant_client.get_collections().collections

In [18]:
collections

[]

In [19]:
COLLECTION_NAME="semantic-search"
VECTOR_SIZE=512

In [20]:
from qdrant_client.http.models import models

In [27]:
if COLLECTION_NAME not in existing_names:
    print(f"Creating collection: {COLLECTION_NAME}")
    qdrant_client.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=models.VectorParams(
            size=VECTOR_SIZE,
            distance=models.Distance.COSINE,
        ),
    )
else:
    print(f"Collection already exists: {COLLECTION_NAME} (reusing)")

Collection already exists: semantic-search (reusing)


In [24]:
collections

[CollectionDescription(name='semantic-search')]

In [26]:
existing_names=[c.name for c in collections]

In [65]:
import numpy as np
import os
from PIL import Image
from uuid import uuid4

In [28]:
def index_image(image_path,category=None):
    img_emb=embedding.embed_image([image_path])[0]
    emb=np.array(img_emb).tolist()
    payload={
        "filename":os.path.basename(image_path),
        "category":category,
        "path":image_path
    }
    qdrant_client.upsert(
        collection_name=COLLECTION_NAME,
        points=[
            models.PointStruct(
                id=str(uuid4()),
                vector=emb,
                payload=payload
            )
        ]
    )
    print(f"Indexed image: {image_path}")
    

In [66]:


def index_folder(root_folder):
    exts = (".jpg", ".jpeg", ".png", ".webp")
    for dirpath, _, files in os.walk(root_folder):
        category = os.path.basename(dirpath)
        for f in files:
            if f.lower().endswith(exts):
                img_path = os.path.join(dirpath, f)
                #print(img_path,category)
                index_image(img_path,category=category)

In [67]:
index_folder("images")

Indexed image: images\animal\cat.jpeg
Indexed image: images\animal\crocodile.jpeg
Indexed image: images\animal\crocodile_1.png
Indexed image: images\animal\dog.jpeg
Indexed image: images\animal\elephant.jpeg
Indexed image: images\animal\giraffe.webp
Indexed image: images\animal\horse.webp
Indexed image: images\animal\lion.jpeg
Indexed image: images\animal\panda.jpg
Indexed image: images\animal\tiger.jpeg
Indexed image: images\animal\zebra.jpeg
Indexed image: images\flower\lavender.jpeg
Indexed image: images\flower\lily.jpeg
Indexed image: images\flower\lotus.jpg
Indexed image: images\flower\marigold.jpeg
Indexed image: images\flower\rose.jpg
Indexed image: images\flower\sunflower.jpeg
Indexed image: images\flower\tulip.webp
Indexed image: images\furniture\table.jpeg
Indexed image: images\general\bottle.jpeg
Indexed image: images\general\car.webp
Indexed image: images\general\chair.jpeg
Indexed image: images\general\cycle.webp
Indexed image: images\general\laptop.jpeg
Indexed image: ima

In [72]:
def search_text(query,top_k=3):
    results=qdrant_client.query_points(
        collection_name=COLLECTION_NAME,
        query=query,
        limit=top_k,
        with_payload=True,
        
    )
    return results

In [69]:
query="A yellow flower"

In [75]:
results=search_text(embedding.embed_query(query))
for point in results.points:
    print(point.payload, "score =", point.score,"vector=",point.vector)

{'filename': 'sunflower.jpeg', 'category': 'flower', 'path': 'images\\flower\\sunflower.jpeg'} score = 0.2515781 vector= None
{'filename': 'marigold.jpeg', 'category': 'flower', 'path': 'images\\flower\\marigold.jpeg'} score = 0.24749102 vector= None
{'filename': 'lily.jpeg', 'category': 'flower', 'path': 'images\\flower\\lily.jpeg'} score = 0.17339094 vector= None


In [76]:
def search_image(image_path,top_k=3):
    emb=embedding.embed_image([image_path])[0]
    results=qdrant_client.query_points(
        collection_name=COLLECTION_NAME,
        query=emb,
        limit=top_k,
        with_payload=True,
        
    )
    return results

In [77]:
cat_image_path=r"C:\Users\Narala Nithin\OneDrive\Desktop\semantic_image_search\semantic_image_search\notebooks\images\animal\cat.jpeg"